### VM setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd "/content/gdrive/MyDrive/Master Thesis/Data/Fuetal2017/"

/content/gdrive/MyDrive/Master Thesis/Data/Fuetal2017


In [3]:
!pip install transformers==4.17

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 4.4 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 880 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 31.7 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=61fb5b424bb8905a09d198c3c23848f609f9fe84152ed1c7be99253b6399c898
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.4 MB/s 
     |████████████████████████████████| 212 kB 11.6 MB/s 
     |████████████████████████████████| 140 kB 46.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.1 MB/s 
     |████████████████████████████████| 127 kB 37.1 MB/s 
     |████████████████████████████████| 271 kB 41.5 MB/s 
     |████████████████████████████████| 144 kB 51.4 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Download git repo from github

In [10]:
!bash ../clone_repo.sh

Cloning into '/content/experiments_chd'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 160 (delta 1), reused 0 (delta 0), pack-reused 151
Receiving objects: 100% (160/160), 24.59 KiB | 94.00 KiB/s, done.
Resolving deltas: 100% (78/78), done.


In [11]:
!bash ../pull_repo.sh
%cd "/content/gdrive/MyDrive/Master Thesis/Data/Fuetal2017/"

Already up to date.
/content/gdrive/MyDrive/Master Thesis/Data/Fuetal2017


### Imports

In [16]:
import sys
sys.path.insert(0, "/content/experiments_chd/data_preparation/")

In [23]:
import glob
from data_loading import ChatHighlightData

ModuleNotFoundError: ignored

In [8]:
import datasets

## Data prep

### Create dataset

#### Load data from published dataset

In [ ]:
!ls

final_data  gt	nalcs_test.txt	nalcs_train.txt  nalcs_val.txt


In [ ]:
# load train, val, test files with filenames
ds_info_files = {"train": "nalcs_train.txt", "validation": "nalcs_val.txt", "test": "nalcs_test.txt"}

In [ ]:
ds_split_files = {"chat": dict(), "highlights": dict()}
for split, file in ds_info_files.items():
  with open(file, "r") as in_file:
    chat, highlights = zip(*[l.rstrip("\n").split(" ")[-2:] for l in in_file.readlines()])
    ds_split_files["chat"][split] = list(chat)
    ds_split_files["highlights"][split] = list(highlights)

In [ ]:
ch_dir = "./final_data/"
hl_dir = "./gt/"

In [ ]:
chd_train = ChatHighlightData(chat_dir=ch_dir, highlight_dir=hl_dir, emote_dir=None, frame_rate=30)
chd_train.load_data(file_identifier="nalcs_w*_g[13]")

In [ ]:
chd_val = ChatHighlightData(chat_dir=ch_dir, highlight_dir=hl_dir, emote_dir=None, frame_rate=30)
chd_val.load_data(file_identifier="nalcs_w[1-4]*_g2")

In [ ]:
chd_test = ChatHighlightData(chat_dir=ch_dir, highlight_dir=hl_dir, emote_dir=None, frame_rate=30)
chd_test.load_data(file_identifier="nalcs_w[5-9]*_g2")

In [ ]:
len(chd_train.chat.keys()), len(chd_eval.chat.keys()), len(chd_test.chat.keys())
# fu et al. 2017 report: 128/40/50

(128, 40, 50)

In [ ]:
# make sure that the chat and highlights match
print(sorted(chd_train.chat.keys()) == sorted(chd_train.highlights.keys()))
print(sorted(chd_val.chat.keys()) == sorted(chd_val.highlights.keys()))
print(sorted(chd_test.chat.keys()) == sorted(chd_test.highlights.keys()))

True
True
True


#### Load data into huggingface datasets, combining them to datasetdict

In [ ]:
def chat_highlight_data_to_huggingface_dataset(chd):

  chat = list()
  highlights = list()
  match_name = list()

  for m, ch in chd.chat.items():
    hl = chd.highlights[m]
    try:
      assert(len(ch) == len(hl))
    except AssertionError:
      print("not matching lengths in:", m)
    name = [m] * len(ch)
    
    chat.extend(ch)
    highlights.extend(hl)
    match_name.extend(name)

  return datasets.Dataset.from_dict({"messages": chat,
                                      "highlights": highlights,
                                      "match_name": match_name
                                    })

In [ ]:
ds_train = chat_highlight_data_to_huggingface_dataset(chd_train)

In [ ]:
ds_val = chat_highlight_data_to_huggingface_dataset(chd_val)

In [ ]:
ds_test = chat_highlight_data_to_huggingface_dataset(chd_test)

In [ ]:
ds = datasets.DatasetDict({"train": ds_train,
                      "val": ds_val,
                      "test": ds_test
                      })

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['messages', 'highlights', 'match_name'],
        num_rows: 10580401
    })
    val: Dataset({
        features: ['messages', 'highlights', 'match_name'],
        num_rows: 3305229
    })
    test: Dataset({
        features: ['messages', 'highlights', 'match_name'],
        num_rows: 3984081
    })
})

In [ ]:
!ls

final_data  gt	nalcs_test.txt	nalcs_train.txt  nalcs_val.txt


In [ ]:
ds.save_to_disk("fu2017_highlight_detection_dataset")

### Reload saved dataset from disk

In [ ]:
ds = datasets.load_from_disk("fu2017_highlight_detection_dataset")

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['messages', 'highlights', 'match_name'],
        num_rows: 10580401
    })
    val: Dataset({
        features: ['messages', 'highlights', 'match_name'],
        num_rows: 3305229
    })
    test: Dataset({
        features: ['messages', 'highlights', 'match_name'],
        num_rows: 3984081
    })
})